In [1]:
import os
import pandas as pd
import numpy as np
import sklearn.metrics
import torch
from bert_sklearn import BertClassifier
from bert_sklearn import BertRegressor
from bert_sklearn import load_model


from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn import preprocessing
from sklearn import metrics
from sklearn import svm

data_df = pd.read_excel('./测试数据.xlsx').sample(n=10000)
data_df = data_df.reset_index(drop=True)
data_df['infos'] = data_df['infos'].apply(lambda x:x.replace('\n','').replace('\t',''))
data_df = data_df[['infos','labels']]
data_df = data_df.drop_duplicates('infos')
data_df.head()


,infos,labels
0,甄嬛提议曹贵人准备和亲的事，为何却要了华妃的性命？,news_entertainment
1,黄奕走红毯，网友：这显得胯真宽，微胖发福了,news_entertainment
2,武汉“新中考”方案出炉，考试科目增至15门，总分增至680分？！,news_edu
3,方舟子说象棋源自印度，中国古代没有大象，你认为这种说法怎么样？,news_culture
4,贾斯汀和泰勒，谁更能吸金？,news_entertainment


In [2]:
np.unique(data_df['labels'].tolist())

array(['news_culture', 'news_edu', 'news_entertainment', 'news_sports'],
      dtype='<U18')

In [3]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(np.unique(data_df['labels'].tolist()))
data_df['labels'] = data_df['labels'].apply(lambda x:label_encoder.transform([x])[0])
data_df.drop_duplicates(inplace=True)
data_df = data_df.reset_index(drop=True)
data_df.head(10)

,infos,labels
0,甄嬛提议曹贵人准备和亲的事，为何却要了华妃的性命？,2
1,黄奕走红毯，网友：这显得胯真宽，微胖发福了,2
2,武汉“新中考”方案出炉，考试科目增至15门，总分增至680分？！,1
3,方舟子说象棋源自印度，中国古代没有大象，你认为这种说法怎么样？,0
4,贾斯汀和泰勒，谁更能吸金？,2
5,在深圳学化妆怎么样，怎么选学校？,1
6,上联：文房四宝千秋志，如何对下联？,0
7,明确告诉你吧，目前市场上大部分太湖石都不是产自太湖！,0
8,上联：河北大地人憨厚，下联怎么对？,0
9,靠自身努力大火的赵丽颖还懂感恩，网友：娱乐圈难得的一股清流！,2


In [4]:
x = data_df['infos'].tolist()
y = data_df['labels'].tolist()

In [15]:
model = BertClassifier()
model.bert_model = "bert-base-cased"
model.epochs = 1
model.use_cuda = False
model.num_labels = 4
model.num_mlp_layers = 1
model.num_mlp_hiddens = 512
model.max_seq_length = 128
model.validation_fraction = 0.15
model.learning_rate = 2e-5
model.gradient_accumulation_steps = 1
model.train_batch_size = 16
model.eval_batch_size = 16

Building sklearn text classifier...


In [16]:
#finetune model
model.fit(x,y)

Loading bert-base-cased model...
Using mlp with D=768,H=512,K=4,n=1
Loading Pytorch checkpoint
train data size: 7749, validation data size: 1367


Training  :   0%|          | 2/484 [00:22<1:41:19, 12.61s/it, loss=1.42]

KeyboardInterrupt: 

In [7]:
model.save('./基础模型.model')

In [8]:
#结果

predictions = model.predict(x)
# result=sklearn.metrics.f1_score(predictions, y,)

Predicting: 100%|██████████| 570/570 [19:05<00:00,  2.01s/it]


In [9]:
result = sklearn.metrics.f1_score(predictions,y,average='micro')
# result=precision_score(predictions, y, average='samples')
print(result)

0.32558139534883723


In [14]:
z = ['恭喜马龙顺利完成乒乓球大满贯！']
pre_z = model.predict(z)
print(pre_z)

Predicting: 100%|██████████| 1/1 [00:08<00:00,  8.59s/it]

[2]
